In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from mrcnn import visualize
from mrcnn.visualize import display_images

"""
Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:
    # Train a new model starting from pre-trained COCO weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=coco
    # Resume training a model that you had trained earlier
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=last
    # Train a new model starting from ImageNet weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=imagenet
    # Apply color splash to an image
    python3 balloon.py splash --weights=/path/to/weights/file.h5 --image=<URL or path to file>
    # Apply color splash to video using the last weights you trained
    python3 balloon.py splash --weights=last --video=<URL or path to file>
"""

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

Using TensorFlow backend.


In [22]:
############################################################
#  Configurations
############################################################

class PostConfig(Config):

    """Configuration for the training
    Derived from the base Config class"""

    NAME = "postit"

    IMAGES_PER_GPU = 2

    NUM_CLASSES = 1 + 1 # Background + postit

    STEPS_PER_EPOCH = 100

    DETECTION_MIN_CONFIDENCE = 0.9

############################################################
#  Dataset
############################################################

class PostDataset(utils.Dataset):
    
    #def load_postit(self, dataset_dir, subset):

    #add class
    #self.add_class("postit", 1, "postit")
    """
    #subset has to be train or val

        assert subset in ["train", "val"]
    """
    dataset_dir = os.path.abspath("../../Coco")
    dataset_dir = os.path.join(dataset_dir, "train")

    #load json file
    #print(dataset_dir)
    data_json = json.load(open(os.path.join(dataset_dir, "output.json")))
    #print(data_json)

    #iterates over all images in the json file

    for i in data_json:
        #print(i['Labeled Data'])

        polygons = [g['geometry'] for g in i['Label']['Post It']]

        image_path = i['Labeled Data']
        image = skimage.io.imread(image_path)
        height, width = image.shape[:2]


        image_id=i['ID']
        """
        self.add_image(
        "postit",
        image_id=i['ID'],
        path=image_path,
        width=width, height=height,
        polygons=polygons
        )
        
        """
        #print(len(polygons))
        mask = np.zeros([height, width, len(polygons)],
                        dtype=np.uint8)
        
        #print(mask)
        
        for i, p in enumerate(polygons):
            x_values = []
            y_values = []
            print(p)
            for x in p:
                x_values.append(x['x'])
                y_values.append(x['y'])

            #print(x_values)
            #print(y_values)
            rr, cc = skimage.draw.polygon(y_values, x_values)
            print(i)

            mask[rr, cc, i] = 1
            #print(mask)

        visualize.display_top_masks(image, mask, 1, "postit")
            

NameError: name 'self' is not defined